In [1]:
# kbo_schema ?

import sqlite3

# Connect to your KBO database
conn = sqlite3.connect('kbo_database.db')
cursor = conn.cursor()

# Get the full CREATE statements for all tables and indices
cursor.execute("SELECT type, name, sql FROM sqlite_master WHERE type IN ('table', 'index', 'view') ORDER BY type, name;")
schema_info = cursor.fetchall()

for obj_type, name, sql in schema_info:
    print(f"-- {obj_type.upper()}: {name}")
    if sql:
        print(sql)
        print()
    else:
        print("(no SQL statement)\n")

# Optionally: List all foreign keys for each table
print("\n--- Foreign Key Details ---")
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [row[0] for row in cursor.fetchall()]
for table in tables:
    cursor.execute(f"PRAGMA foreign_key_list('{table}')")
    fks = cursor.fetchall()
    if fks:
        print(f"\nTable: {table}")
        for fk in fks:
            print(f"  {fk}")

conn.close()


-- INDEX: 
CREATE INDEX "" ON "address" ("EntityNumber")

-- INDEX: EnterpriseNumber
CREATE INDEX "EnterpriseNumber" ON "establishment" ("EnterpriseNumber")

-- INDEX: EntityNumber
CREATE INDEX "EntityNumber" ON "activity" ("EntityNumber")

-- INDEX: EstablishmentNumber
CREATE INDEX "EstablishmentNumber" ON "establishment" ("EstablishmentNumber")

-- INDEX: branch_EnterpriseNumber_idx
CREATE INDEX "branch_EnterpriseNumber_idx" ON "branch" ("EnterpriseNumber")

-- INDEX: code_Code_idx
CREATE INDEX "code_Code_idx" ON "code" ("Code")

-- INDEX: contact_EntityNumber_idx
CREATE INDEX "contact_EntityNumber_idx" ON "contact" ("EntityNumber")

-- INDEX: denomination_EntityNumber_idx
CREATE INDEX "denomination_EntityNumber_idx" ON "denomination" ("EntityNumber")

-- INDEX: enterprise_EnterpriseNumber_idx
CREATE INDEX "enterprise_EnterpriseNumber_idx" ON "enterprise" ("EnterpriseNumber")

-- TABLE: activity
CREATE TABLE "activity" (
"EntityNumber" TEXT,
  "ActivityGroup" INTEGER,
  "NaceVersion"

In [ ]:
More info...

In [2]:
import sqlite3

db_path = 'kbo_database.db'
conn = sqlite3.connect(db_path)
cur = conn.cursor()

print("=== Tables and Columns ===")
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [r[0] for r in cur.fetchall()]

for table in tables:
    print(f"\nTable: {table}")
    cur.execute(f"PRAGMA table_info('{table}')")
    for col in cur.fetchall():
        print(f"  - {col[1]} ({col[2]})")  # col[1]=name, col[2]=type

print("\n=== Indices (PK/Unique/Other) ===")
cur.execute("SELECT name FROM sqlite_master WHERE type='index';")
indices = [r[0] for r in cur.fetchall()]

for idx in indices:
    print(f"\nIndex: {idx}")
    cur.execute(f"PRAGMA index_info('{idx}')")
    for info in cur.fetchall():
        print(f"  - column: {info[2]}")
    # Check if the index is unique
    cur.execute(f"PRAGMA index_list('{tables[0]}')")
    idx_list = cur.fetchall()
    for i in idx_list:
        if i[1] == idx:
            print(f"  - Unique: {'Yes' if i[2] else 'No'}")

conn.close()


=== Tables and Columns ===

Table: denomination
  - EntityNumber (TEXT)
  - Language (INTEGER)
  - TypeOfDenomination (INTEGER)
  - Denomination (TEXT)

Table: branch
  - Id (TEXT)
  - StartDate (TEXT)
  - EnterpriseNumber (TEXT)

Table: activity
  - EntityNumber (TEXT)
  - ActivityGroup (INTEGER)
  - NaceVersion (INTEGER)
  - NaceCode (INTEGER)
  - Classification (TEXT)

Table: establishment
  - EstablishmentNumber (TEXT)
  - StartDate (TEXT)
  - EnterpriseNumber (TEXT)

Table: address
  - EntityNumber (TEXT)
  - TypeOfAddress (TEXT)
  - CountryNL (TEXT)
  - CountryFR (TEXT)
  - Zipcode (TEXT)
  - MunicipalityNL (TEXT)
  - MunicipalityFR (TEXT)
  - StreetNL (TEXT)
  - StreetFR (TEXT)
  - HouseNumber (TEXT)
  - Box (TEXT)
  - ExtraAddressInfo (TEXT)
  - DateStrikingOff (TEXT)

Table: contact
  - EntityNumber (TEXT)
  - EntityContact (TEXT)
  - ContactType (TEXT)
  - Value (TEXT)

Table: meta
  - Variable (TEXT)
  - Value (TEXT)

Table: code
  - Category (TEXT)
  - Code (TEXT)
  - Langu

In [3]:
import sqlite3
conn = sqlite3.connect('kbo_database.db')
cur = conn.cursor()
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()
print(tables)
conn.close()


[('denomination',), ('branch',), ('activity',), ('establishment',), ('address',), ('contact',), ('meta',), ('code',), ('enterprise',)]


Summary of Your Database Structure
Key Tables and Their Main Columns:
enterprise:

EnterpriseNumber (acts as unique ID for companies)

establishment:

EstablishmentNumber (branch ID), EnterpriseNumber (links to company)

branch:

Id, EnterpriseNumber

activity, address, denomination, contact:

All have EntityNumber (link field)

code:

Category, Code, Language, Description

meta:

Metadata about the dataset

Indices:
Indices on almost every linking field (e.g., EnterpriseNumber, EntityNumber, EstablishmentNumber)

No explicit PRIMARY KEY or FOREIGN KEY, but indices act as the relational "glue".

Example:

Index on EntityNumber in denomination, contact, activity, and address tables

Index on EnterpriseNumber in enterprise, establishment, branch

Index on EstablishmentNumber in establishment

Index on Code in code

How to Join / Analyze the Tables
Typical join logic:

To get all info about a company:

Join tables on EnterpriseNumber (main ID for companies)

To get all info about an establishment/branch:

Use EstablishmentNumber, then join to enterprise via EnterpriseNumber

To get all addresses, activities, contacts, denominations for a company:

Join via EntityNumber (assume this is same as EnterpriseNumber for those records)



In [8]:
!pip install jupysql

In [9]:
%load_ext sql
%sql sqlite:///kbo_database.db

Connecting to 'sqlite:///kbo_database.db'

In [10]:
%%sql

-- A. Get all company names and their addresses
SELECT e.EnterpriseNumber, a.StreetNL, a.MunicipalityNL, a.Zipcode
FROM enterprise e
LEFT JOIN address a ON e.EnterpriseNumber = a.EntityNumber
LIMIT 20;


Running query in 'sqlite:///kbo_database.db'

EnterpriseNumber,StreetNL,MunicipalityNL,Zipcode
0200.065.765,Panhuisstraat,Destelbergen,9070
0200.068.636,Stropstraat,Gent,9000
0200.171.970,Brabantdam,Gent,9000
0200.245.711,Hoge Buizemont,Geraardsbergen,9500
0200.305.493,Gentsesteenweg,Sint-Lievens-Houtem,9520
0200.362.210,Rue de la Religion,Nivelles,1400
0200.362.408,Rue du Cerf,Rixensart,1332
0200.420.410,Michel Theysstraat,Diest,3290
0200.420.608,Ieperstraat,Poperinge,8970
0200.448.421,Budastraat(Kor),Kortrijk,8500


In [15]:
%%sql

-- B. List all establishments (branches) for a given company
SELECT est.EstablishmentNumber, est.StartDate
FROM establishment est
WHERE est.EnterpriseNumber = '0200.065.765';




Running query in 'sqlite:///kbo_database.db'

EstablishmentNumber,StartDate
2.251.273.097,01-12-2015


In [18]:
%%sql

-- C. Show all activities for a company
SELECT act.NaceCode, act.Classification
FROM activity act
WHERE act.EntityNumber = '0200.065.765';

Running query in 'sqlite:///kbo_database.db'

NaceCode,Classification
84130,MAIN
70111,MAIN
68121,MAIN
84130,MAIN
41101,MAIN


In [19]:
%%sql

-- D. List all contacts for a company
SELECT c.ContactType, c.Value
FROM contact c
WHERE c.EntityNumber = '0200.065.765';



Running query in 'sqlite:///kbo_database.db'

ContactType,Value


In [20]:
%%sql

-- E. Find all companies in a specific postal code
SELECT e.EnterpriseNumber, a.Zipcode
FROM enterprise e
LEFT JOIN address a ON e.EnterpriseNumber = a.EntityNumber
WHERE a.Zipcode = '1000';

Running query in 'sqlite:///kbo_database.db'

EnterpriseNumber,Zipcode
0200.882.050,1000
0202.962.701,1000
0203.071.973,1000
0203.201.340,1000
0203.211.040,1000
0203.286.759,1000
0203.375.049,1000
0206.048.091,1000
0206.492.907,1000
0206.694.528,1000
